<a href="https://colab.research.google.com/github/cpyp-ipdp/pgd_cdmx/blob/main/indicadores/vivienda/indicadores_vivienda_pgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Estimación de indicadores de vivienda para el Plan General de Desarrollo de la Ciudad de México

En este script se descargan los microdatos de la Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) para los años 2016, 2018, 2020 y 2022. Posteriormente, se estiman dos indicadores de vivienda para la Ciudad de México: rezago habitacional y viviendas sin escrituras. Los resultados se presentan en dos cuadros.

In [1]:
##Estimación de indicadores de vivienda
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr,archive)
remotes::install_github("cpyp-ipdp/funcionespgd", force = TRUE)
library(funcionespgd)
font_add_google("Poppins", "pop")
showtext_auto()

Cargando paquete requerido: pacman

Using GitHub PAT from the git credential store.




rlang  (1.1.4 -> 1.1.6) [CRAN]
glue   (1.7.0 -> 1.8.0) [CRAN]
cli    (3.6.3 -> 3.6.5) [CRAN]
tibble (3.2.1 -> 3.3.0) [CRAN]


Installing 4 packages: rlang, glue, cli, tibble

Warning message:
"package 'tibble' is in use and will not be installed"
Installing packages into 'C:/Users/claud/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'rlang' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rlang'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problema al copiar C:\Users\claud\AppData\Local\R\win-library\4.4\00LOCK\rlang\libs\x64\rlang.dll  a C:\Users\claud\AppData\Local\R\win-library\4.4\rlang\libs\x64\rlang.dll: Permission denied"
Warning message:
"restored 'rlang'"


...

Warning message:
"cannot remove prior installation of package 'glue'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problema al copiar C:\Users\claud\AppData\Local\R\win-library\4.4\00LOCK\glue\libs\x64\glue.dll  a C:\Users\claud\AppData\Local\R\win-library\4.4\glue\libs\x64\glue.dll: Permission denied"
Warning message:
"restored 'glue'"


package 'cli' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'cli'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problema al copiar C:\Users\claud\AppData\Local\R\win-library\4.4\00LOCK\cli\libs\x64\cli.dll  a C:\Users\claud\AppData\Local\R\win-library\4.4\cli\libs\x64\cli.dll: Permission denied"
Warning message:
"restored 'cli'"



The downloaded binary packages are in
	C:\Users\claud\AppData\Local\Temp\RtmpieDikO\downloaded_packages
── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file 'C:\Users\claud\AppData\Local\Temp\RtmpieDikO\remotes18d45016d6a\cpyp-ipdp-funcionespgd-6fd64b9/DESCRIPTION' ... OK
* preparing 'funcionespgd':
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted 'LazyData' from DESCRIPTION
* building 'funcionespgd_0.1.0.tar.gz'



Installing package into 'C:/Users/claud/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin<-"_ns_viviendas_csv.zip"

In [4]:
years<-c("2016", "2018", "2020", "2022","2024")

In [5]:
# Descargar los archivos, descomprimirlos y renombrarlos
for (i in 1:length(years)) {
  url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
  destfile <- paste0("data/", years[i], fin)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = "data")

  # Renombrar el archivo descomprimido a "viviendas_xxxx.csv"
  file.rename(
    from = "data/viviendas.csv",
    to = paste0("data/viviendas_", years[i], ".csv")
  )
}

In [6]:
#Enlistar archivos de la carpeta data
list.files("data")

[1] "2016_ns_viviendas_csv.zip"         "2018_ns_viviendas_csv.zip"        
 [3] "2020_ns_viviendas_csv.zip"         "2022_ns_viviendas_csv.zip"        
 [5] "2024_ns_viviendas_csv.zip"         "nota_bases_datos_enigh2016_ns.txt"
 [7] "nota_bases_datos_enigh2018_ns.txt" "viviendas_2016.csv"               
 [9] "viviendas_2018.csv"                "viviendas_2020.csv"               
[11] "viviendas_2022.csv"                "viviendas_2024.csv"

### Función de estimación

In [7]:
estimacion <- function(path, variable) {
  # Cargar los datos
  print(path)
  datos <- read_csv(path)
  valor_excusado <- if (length(unique(datos$excusado)) == 3) 3 else 2

  datos <- datos %>%
    mutate(
      across(starts_with("mat") & where(is.character),
             ~ replace_na(parse_number(.x, na = c('', 'NA', '&')), 0)),
      rezago = if_else(
        ((tot_resid / num_cuarto) > 2.5) |
          (mat_pared %in% 1:6) |
          (mat_techos %in% c(1:4, 6, 7, 9)) |
          (mat_pisos == 1) |
          (excusado == valor_excusado),
        "En rezago",
        "Fuera de rezago"
      )
    ) %>%
    mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")


  # Definir diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estimar rezago habitacional
  rezago <- mydesign %>%
    group_by(rezago) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
        #Viviendas totales
    ) %>%
    mutate(pct = round(pct * 100, 2)) %>%
  # Añadir columna con el total de viviendas
  mutate(total_viviendas = sum(viviendas)) %>%
    filter(rezago == "En rezago") %>%
    # Añadir año
    mutate(year = substr(path, 16, 19))




  # Estimar viviendas sin escrituras
  escrituras <- mydesign %>%
    filter(tenencia %in% c(3,4)) %>%
group_by(escrituras) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
    ) %>%
 mutate(total_viviendas = sum(viviendas)) %>%
    filter(escrituras == 3) %>%
    mutate(pct = round(pct * 100, 2),
           escrituras = case_when(
        escrituras == 3 ~ "Sin escrituras"
))%>%
mutate(year = substr(path, 16, 19))


#Hogares en la vivienda

hogares<-mydesign %>%
summarise(
    hogares=survey_total(tot_hog, vartype = "cv")
    ) %>%
mutate(year = substr(path, 16, 19))


##Hogares hacinamiento

hogares_hac<-mydesign %>%
#Identificar hacinamiento
mutate(hacinamiento=ifelse((tot_resid/num_cuarto)>2.5,1,0))%>%
#filtrar hacinamiento
#filter(hacinamiento==1)%>%
group_by(hacinamiento)%>%
summarise(
    hogares_hac=survey_total(vartype = "cv"),
    pct=survey_prop(vartype = "cv")
    ) %>%
mutate(year = substr(path, 16, 19))%>%

mutate(pct=round(pct*100,2))


hogares_hac2<-mydesign %>%
#Identificar hacinamiento
mutate(hacinamiento=ifelse((tot_resid/num_cuarto)>2,1,0))%>%
#filtrar hacinamiento
#filter(hacinamiento==1)%>%
group_by(hacinamiento)%>%
summarise(
    hogares_hac=survey_total(vartype = "cv"),
    pct=survey_prop(vartype = "cv")
    ) %>%
mutate(year = substr(path, 16, 19))%>%
mutate(pct=round(pct*100,2))


#Hog hacinamiento no propietarios
hogares_hac_no_prop<-mydesign%>%
#Identificar hacinamiento
mutate(hacinamiento=ifelse((tot_resid/num_cuarto)>2.5,1,0))%>%
mutate(ten_vivi=case_when(tenencia==3 | tenencia==4 ~ "Propietarios",
            TRUE ~ "No propietarios"))%>%
#filtrar hacinamiento
filter(hacinamiento==1)%>%
group_by(ten_vivi)%>%
#Contar hogares con hacinamiento
summarise(
    hogares_hac=survey_prop(vartype = "cv")
    ) %>%
ungroup()%>%
mutate(year = as.integer(substr(path, 16, 19)))%>%
filter(ten_vivi=="No propietarios")%>%
mutate(hogares_hac=round(hogares_hac*100,2))



  # Devolver el resultado basado en la variable
  if (variable == "rezago") {
    return(rezago)

  } else if (variable == "sin_escrituras") {
    return(escrituras)
  }

else if (variable == "hogares") {
    return(hogares)
  }

  else if (variable == "hogares_hac_no_prop") {
    return(hogares_hac_no_prop)
}

else if (variable == "hogares_hac") {
    return(hogares_hac)

  }
    else if (variable == "hogares_hac2") {
        return(hogares_hac2)

    }

else {
    stop("Variable no reconocida. Use 'rezago' o 'sin_escrituras'.")
  }}
     

In [8]:
#Construir cuadros de rezago habitacional

tabla_rezago<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
  
  estimacion,
  variable = "rezago"
)%>%
mutate(year = as.integer(year))
print("Porcentaje de viviendas cuyos habitantes viven en rezago habitacional")
tabla_rezago

[1] "data/viviendas_2016.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2018.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2020.csv"


...

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 88823 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2024.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 90324 Columns: 82
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): folioviv, mat_techos, finan_8, num_dueno1, num_dueno2, ubica_geo, ...
dbl (74): tipo_viv, mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Porcentaje de viviendas cuyos habitantes viven en rezago habitacional"


rezago,viviendas,viviendas_cv,pct,pct_cv,total_viviendas,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
En rezago,272200,0.08809435,9.81,0.08684886,2774780,2016
En rezago,218501,0.08906203,8.02,0.08727815,2724389,2018
En rezago,198021,0.08360376,7.31,0.08292165,2707156,2020
En rezago,175242,0.09933936,5.94,0.09870452,2947953,2022
En rezago,182009,0.09643562,6.01,0.09677589,3029724,2024


In [9]:
#Construir cuadros de viviendas sin escrituras
#Viviendas sin escrituras corresponden a viviendas propias y propias que se están pagando y no cuentan con escrituras

tabla_escrituras<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable = "sin_escrituras"
    )%>%
    mutate(
  year = as.integer(year)
)
print("Porcentaje de viviviendas propias o propias en proceso de pago que no cuentan con escrituras")
tabla_escrituras

[1] "data/viviendas_2016.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2018.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2020.csv"


...

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 88823 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2024.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 90324 Columns: 82
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): folioviv, mat_techos, finan_8, num_dueno1, num_dueno2, ubica_geo, ...
dbl (74): tipo_viv, mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Porcentaje de viviviendas propias o propias en proceso de pago que no cuentan con escrituras"


escrituras,viviendas,viviendas_cv,pct,pct_cv,total_viviendas,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Sin escrituras,266547,0.08646642,17.71,0.08298775,1505322,2016
Sin escrituras,246283,0.09985847,17.22,0.09461881,1429942,2018
Sin escrituras,259056,0.09464920,16.23,0.08765464,1595765,2020
Sin escrituras,310191,0.08428046,17.63,0.07835935,1759030,2022
Sin escrituras,294660,0.09017554,16.48,0.08552691,1787817,2024


In [10]:
tabla_hogares<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable = "hogares"
    )%>%
mutate(
  year = as.integer(year),
tasa_crec=(hogares/lag(hogares)-1)*100)
tabla_hogares

[1] "data/viviendas_2016.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2018.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2020.csv"


...

[1] "data/viviendas_2022.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 88823 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2024.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 90324 Columns: 82
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): folioviv, mat_techos, finan_8, num_dueno1, num_dueno2, ubica_geo, ...
dbl (74): tipo_viv, mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hogares,hogares_cv,year,tasa_crec
<dbl>,<dbl>,<int>,<dbl>
2825100,0.01357273,2016,NA
2778842,0.01510168,2018,-1.637393
2731683,0.01285271,2020,-1.697074
2990030,0.01388237,2022,9.457430
3082330,0.01299118,2024,3.086926


In [11]:
tabla_hogares_hac<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable="hogares_hac"
    )%>%
mutate(
  year = as.integer(year),
  #asa_crec = (hogares_hac / lag(hogares_hac) - 1) * 100
)
#filter(hacinamiento==1)
print("Porcentaje de hogares en hacinamiento")
tabla_hogares_hac

[1] "data/viviendas_2016.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2018.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2020.csv"


...

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 88823 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2024.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 90324 Columns: 82
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): folioviv, mat_techos, finan_8, num_dueno1, num_dueno2, ubica_geo, ...
dbl (74): tipo_viv, mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Porcentaje de hogares en hacinamiento"


hacinamiento,hogares_hac,hogares_hac_cv,pct,pct_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0,2680313,0.01415185,96.60,0.005249547,2016
1,94467,0.14946741,3.40,0.148945447,2016
0,2654945,0.01471695,97.45,0.004330904,2018
1,69444,0.16685083,2.55,0.165576735,2018
0,2627298,0.01316224,97.05,0.003695403,2020
1,79858,0.12260726,2.95,0.121577353,2020
0,2897338,0.01401708,98.28,0.002838060,2022
1,50615,0.16253584,1.72,0.162458155,2022
0,2965930,0.01355190,97.89,0.003440907,2024


In [12]:
tabla_hogares_hac2<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable="hogares_hac2"
    )%>%
mutate(
  year = as.integer(year),
  #asa_crec = (hogares_hac / lag(hogares_hac) - 1) * 100
)
print("Porcentaje de hogares en hacinamiento considerando relación de 2 o más personas por cuarto")
tabla_hogares_hac2

[1] "data/viviendas_2016.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2018.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2020.csv"


...

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 88823 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "data/viviendas_2024.csv"


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 90324 Columns: 82
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): folioviv, mat_techos, finan_8, num_dueno1, num_dueno2, ubica_geo, ...
dbl (74): tipo_viv, mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, co...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Porcentaje de hogares en hacinamiento considerando relación de 2 o más personas por cuarto"


hacinamiento,hogares_hac,hogares_hac_cv,pct,pct_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0,2636404,0.01500624,95.01,0.006614794,2016
1,138376,0.12600871,4.99,0.126028132,2016
0,2610569,0.01520139,95.82,0.005401385,2018
1,113820,0.12485900,4.18,0.123885849,2018
0,2583827,0.01340693,95.44,0.004402960,2020
1,123329,0.09333855,4.56,0.092245027,2020
0,2865365,0.01424585,97.20,0.003658250,2022
1,82588,0.12711455,2.80,0.126921833,2022
0,2933239,0.01374539,96.82,0.004059070,2024


In [13]:
# tabla_hogares_hac_no_prop<-map_dfr(
#     list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
#     estimacion,
#     variable="hogares_hac_no_prop"
#     )
# print("Porcentaje de hogares en hacinamiento que no son propietarios de la vivienda")
# tabla_hogares_hac_no_prop

### Indicador de migración por causas relacionadas con ubicación o vivienda


Para este indicador se utilizan los datos de migración del Censo. Concretamente, se consideran aquellos que cambiaron de residencia en los últimos cinco años y que se mudaron por causas relacionadas con ubicación o vivienda.

In [14]:
#Descarga de población de Censo 2020

# url_censo<-"https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_eum_csv.zip"

In [15]:
# # Verificar si existe el archivo Personas00.CSV
# if (!file.exists("data/Personas00.CSV")) {
#   # Descarga en carpeta data
#   destfile_censo <- "data/Censo2020_CA_eum_csv.zip"

#   # Configuración de tiempo de espera para la descarga
#   options(timeout = 900)
#   download.file(url_censo, destfile = destfile_censo, quiet = FALSE, mode = "wb")

#   # Ruta a WinRAR (asegúrate de que sea la correcta)
#   winrar <- shQuote("C:/Program Files/WinRAR/WinRAR")
#   destino <- paste0(getwd(), "/data/")

#   # Ejecutar extracción
#   cmd <- paste(winrar, 'e', shQuote(destfile_censo), '-r', shQuote(destino), sep = ' ')
#   system(cmd)
# } else {
#   message("El archivo Personas00.CSV ya existe en la carpeta data. Proceso omitido.")
# }

In [16]:
# pob<-read_csv("data/Personas00.CSV")%>%
# janitor::clean_names()%>%
# filter(ent_pais_res_5a <= "032") %>%
# #Sustraer 2 dígitos de la clave de la entidad
# mutate(ent_pais_res_5a = substr(ent_pais_res_5a, 2, 3))%>%
#    mutate(ent_pais_res_5a = case_when(
#      nchar(as.character(ent_pais_res_5a)) == 1 ~ paste0("0", ent_pais_res_5a),
#      TRUE ~ as.character(ent_pais_res_5a)
#    )) %>%

#   mutate(ent = case_when(
#   nchar(as.character(ent)) == 1 ~ paste0("0", ent),TRUE ~ as.character(ent)))%>%
#  # No contabilizar la misma entidad
#     filter(ent_pais_res_5a=="09")%>%
# filter(ent!="09")%>%
#   filter(edad>=5)%>%

# ##Se identifican las causas de migración relacionadas con la vivienda y ubicación
# #El catálogo de causas se puede descargar de https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/702825198701.pdf

# mutate(causa_vivi=
# case_when(causa_mig_v %in% c("0104","0201","0503","0901","0902", "0903","0905","0909") ~ "Causas relacionadas con ubicación o vivienda",
# TRUE ~ "Otra causa"))

In [17]:
# #Opción para ajustar si se tiene una sola upm
# options(survey.lonely.psu="certainty")
# #Definir diseño muestral

# dm<-pob%>%
#   #Diseño muestral
#   as_survey_design(weights=factor, strata=estrato, ids=upm, nest=TRUE)

In [18]:
# print("Personas que salieron de CDMX de 2015 a 2020 por causas relacionadas con la ubicación o vivienda")
# dm%>%
# group_by(causa_vivi)%>%
#     summarise(personas=survey_total(vartype="cv"),
#               porcentaje=survey_prop(vartype="cv"))%>%
#     mutate(porcentaje=round(porcentaje*100,2))

#### Indicador de pobreza por calidad y espacios de la vivienda de Evalúa CDMX

In [19]:

# Librerías ====
# if(!require('pacman')) install.packages('pacman')
# pacman::p_load(tidyverse, srvyr, haven, googledrive)

In [20]:
#Crear directorio para guardar los archivos
# dir.create("data", showWarnings = FALSE)

In [21]:
  #Conectar a google drive
# drive_auth(email = "claudiodanielpc@gmail.com",cache = FALSE)

In [22]:
# carpeta<-as_id(drive_get("https://drive.google.com/drive/folders/1TBfoik4w4DWhwFqWeZFgkZvrBdDbqzku"))


# archivos<-drive_ls(carpeta)
# print(archivos)

In [23]:
# estimacion_pobreza <- function(file_id, file_name) {
#   # Descarga de datos
#   local_path <- file.path("data", file_name)
#   drive_download(as_id(file_id), path = local_path, overwrite = TRUE)

#   # Leer datos
#   datos <- read_dta(local_path)

#   # Añadir año
#   year <- str_extract(file_name, "\\d{4}")


#   # Diseño muestral
#   mydesign <- datos %>%
#     as_survey_design(ids = upm,
#                      strata = est_dis,
#                      weights = factor)

#   # Estima pobreza
#   pobreza <- mydesign %>%
#     #Filtrar datos para CDMX
#   filter(entidad==9)%>%
#     #Filtar grupo de población en pobreza
#    group_by(pobreza_ccevj)%>%
#     summarise(
#       poblacion = survey_total(vartype = "cv"),
#       porcentaje = survey_prop(vartype = "cv")
#     ) %>%
#     mutate(porcentaje = round(porcentaje * 100, 2),
#            year = as.numeric(year))%>%
#             filter(pobreza_ccevj == 1)

#   return(pobreza)
# }

In [24]:
#tabla_pobreza <- map_dfr(
#  seq_len(nrow(archivos)),
#  ~ estimacion_pobreza(archivos$id[.x], archivos$name[.x])
#)%>%
#  arrange(year)

#print("Estimación de población en situación de pobreza por calidad y espacios de la vivienda en la CDMX")
#tabla_pobreza

#### Proyecciones a 2045

Se seleccionan dos indicadores: porcentaje de rezago habitacional y tenencia de escrituras

In [25]:
tabla_rezago

rezago,viviendas,viviendas_cv,pct,pct_cv,total_viviendas,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
En rezago,272200,0.08809435,9.81,0.08684886,2774780,2016
En rezago,218501,0.08906203,8.02,0.08727815,2724389,2018
En rezago,198021,0.08360376,7.31,0.08292165,2707156,2020
En rezago,175242,0.09933936,5.94,0.09870452,2947953,2022
En rezago,182009,0.09643562,6.01,0.09677589,3029724,2024


In [26]:
tcma_rezago<-((tabla_rezago$pct[tabla_rezago$year==2022]/tabla_rezago$pct[tabla_rezago$year==2016])^(1/(2022-2016))-1)

tcma_rezago

[1] -0.08021518

In [27]:
extendido_rezago<-data.frame(year=2025:2045)
extendido_rezago$pct<-tabla_rezago$pct[tabla_rezago$year==2024]*cumprod(rep(1+tcma_rezago,length(2025:2045)))
extendido_rezago%>%
mutate(pct=round(pct,2))

year,pct
<int>,<dbl>
2025,5.53
2026,5.08
2027,4.68
2028,4.30
2029,3.96
2030,3.64
2031,3.35
2032,3.08
2033,2.83


In [28]:
tabla_escrituras

escrituras,viviendas,viviendas_cv,pct,pct_cv,total_viviendas,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Sin escrituras,266547,0.08646642,17.71,0.08298775,1505322,2016
Sin escrituras,246283,0.09985847,17.22,0.09461881,1429942,2018
Sin escrituras,259056,0.09464920,16.23,0.08765464,1595765,2020
Sin escrituras,310191,0.08428046,17.63,0.07835935,1759030,2022
Sin escrituras,294660,0.09017554,16.48,0.08552691,1787817,2024


In [29]:
tcma_esc<-((tabla_escrituras$pct[tabla_escrituras$year==2020]/tabla_escrituras$pct[tabla_escrituras$year==2016])^(1/(2022-2016))-1)

tcma_esc

[1] -0.01443942

In [30]:
extendido_escrituras<-data.frame(year=2025:2045)
extendido_escrituras$pct<-tabla_escrituras$pct[tabla_escrituras$year==2024]*cumprod(rep(1+tcma_esc,length(2025:2045)))
extendido_escrituras%>%
mutate(pct=round(pct,2))

year,pct
<int>,<dbl>
2025,16.24
2026,16.01
2027,15.78
2028,15.55
2029,15.32
2030,15.10
2031,14.88
2032,14.67
2033,14.46


In [31]:
tcma_hoghac<-((tabla_hogares_hac$pct[tabla_hogares_hac$year==2022]/tabla_hogares_hac$pct[tabla_hogares_hac$year==2016])^(1/(2022-2016))-1)

tcma_hoghac

[1]  0.002877767 -0.107362924

### Escenarios

In [32]:
configuraciones <- tibble::tibble(
  tabla_obs = list(tabla_rezago, tabla_escrituras),
  tabla_est = list(extendido_rezago, extendido_escrituras),
  margen = c(0.015, 0.015),
    base=c(2024,2024),
  linea_base=c(2024,2024),
      despegue = list(
    c(1, 0),  
    c(0.8, 0.3) 
  ),
  nombre_archivo = c("rezago_estimada.png",
                     "escrituras_estimada.png"),
  titulo = c("Porcentaje de viviendas en rezago habitacional",
             "Porcentaje de viviendas propias 
o propias en proceso de pago que no cuentan con escrituras"),
  nombre_estimado_futuro = c("Deseable", "Deseable"),
  nombre_intervalo_superior = c("Inercial", "Inercial"),
  nombre_intervalo_inferior = c("Transformador", "Transformador"),
  limite_inferior_y = c(0,8),
  variable_intervals = c("pct", "pct")
)

In [33]:
generar_grafica <- function(tabla_obs, tabla_est, margen, titulo,base, linea_base,
                            nombre_archivo, nombre_estimado_futuro,despegue,
                            nombre_intervalo_superior, nombre_intervalo_inferior,
                            limite_inferior_y, variable_intervals = "porcentaje") {

  serie_completa <- dplyr::bind_rows(
    tabla_obs %>%
      dplyr::mutate(year = as.numeric(year)) %>%
      intervals(variable_intervals, margen = 0, año_base = base),

    tabla_est %>%
      dplyr::mutate(year = as.numeric(year)) %>%
      intervals(variable_intervals, margen = margen, año_base = base)
  )

  grafica <- grafica_bonita(
    data = serie_completa,
    x = "year",
    y = "estimado",
    titulo = titulo,
        ano_base=base,
     mostrar_etiqueta_ano_base = TRUE,
    linea_vertical = linea_base,
    anios_etiquetas=c(2045),
      desplazamiento_ano_base = despegue,
    mostrar_intervalo = "ambos",
    nombre_observado = "Observado",
    nombre_estimado_futuro = nombre_estimado_futuro,
    nombre_intervalo_superior = nombre_intervalo_superior,
    nombre_intervalo_inferior = nombre_intervalo_inferior,
    mostrar_leyenda = TRUE,
    fuente = "pop",
    etiqueta_x = "Año",
    etiqueta_y = "Porcentaje",
    titulo_leyenda = "Escenarios",
    limite_inferior_y = limite_inferior_y
  )

  ggplot2::ggsave(paste0("H:/Mi unidad/pgd/graficas/",nombre_archivo), grafica, 
  width = 10, height = 6, dpi = 185, bg = "white")
}


In [34]:
pmap(
  configuraciones,
  function(tabla_obs, tabla_est, margen, nombre_archivo,base,linea_base,
           titulo, nombre_estimado_futuro,despegue,
           nombre_intervalo_superior, nombre_intervalo_inferior,
           limite_inferior_y, variable_intervals) {
    generar_grafica(
      tabla_obs = tabla_obs,
      tabla_est = tabla_est,
      margen = margen,
      base=base,
      linea_base=linea_base,
       despegue = despegue,
      titulo = titulo,
      nombre_archivo = nombre_archivo,
      nombre_estimado_futuro = nombre_estimado_futuro,
      nombre_intervalo_superior = nombre_intervalo_superior,
      nombre_intervalo_inferior = nombre_intervalo_inferior,
      limite_inferior_y = limite_inferior_y,
      variable_intervals = variable_intervals
    )
  }
)

[[1]]
[1] "H:/Mi unidad/pgd/graficas/rezago_estimada.png"

[[2]]
[1] "H:/Mi unidad/pgd/graficas/escrituras_estimada.png"